## Deliverable 2. Create a Customer Travel Destinations Map.

In [29]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
import matplotlib.pyplot as plt
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Laguna,38.4210,-121.4238,35.25,28,1,2.68,US,2021-06-06 22:34:27
1,1,Hobart,-42.8794,147.3294,13.48,77,75,2.68,AU,2021-06-06 22:34:07
2,2,Peleduy,59.6289,112.7406,8.95,66,45,3.47,RU,2021-06-06 22:34:55
3,3,Bambous Virieux,-20.3428,57.7575,24.14,78,40,6.45,MU,2021-06-06 22:34:57
4,4,Novo Aripuanã,-5.1206,-60.3797,26.12,78,97,0.59,BR,2021-06-06 22:34:59


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
print("Enter the maximum preffered temperature for your travel destination:")
max_t = input()

print("Enter the minimum preffered temperature for your travel destination:")
min_t = input()



Enter the maximum preffered temperature for your travel destination:
Enter the minimum preffered temperature for your travel destination:


In [3]:
max_t = 32
min_t = 20

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_t) & \
                                       (city_data_df["Max Temp"] >= min_t)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,Bambous Virieux,-20.3428,57.7575,24.14,78,40,6.45,MU,2021-06-06 22:34:57
4,4,Novo Aripuanã,-5.1206,-60.3797,26.12,78,97,0.59,BR,2021-06-06 22:34:59
7,7,Jamestown,42.0970,-79.2353,28.62,50,1,3.09,US,2021-06-06 22:34:15
19,19,Cayenne,4.9333,-52.3333,26.16,86,0,2.45,GF,2021-06-06 22:35:28
24,24,Fukuechō,32.6881,128.8419,21.18,77,100,2.82,JP,2021-06-06 22:35:37
28,28,Sittwe,20.1500,92.9000,28.57,83,100,3.00,MM,2021-06-06 22:35:45
30,30,Srīvardhan,18.0333,73.0167,27.80,76,37,2.35,IN,2021-06-06 22:35:48
34,34,Alofi,-19.0595,-169.9187,26.94,61,54,3.60,NU,2021-06-06 22:35:55
36,36,Vaini,-21.2000,-175.2000,25.09,78,75,6.17,TO,2021-06-06 22:34:02
39,39,Butaritari,3.0707,172.7902,28.10,72,12,3.79,KI,2021-06-06 22:36:08


In [5]:
# 4a. Determine if there are any empty rows.
[preferred_cities_df.iloc[i,j] for i,j in zip(*np.where(pd.isnull(preferred_cities_df)))]

[nan]

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,Bambous Virieux,-20.3428,57.7575,24.14,78,40,6.45,MU,2021-06-06 22:34:57
4,4,Novo Aripuanã,-5.1206,-60.3797,26.12,78,97,0.59,BR,2021-06-06 22:34:59
7,7,Jamestown,42.0970,-79.2353,28.62,50,1,3.09,US,2021-06-06 22:34:15
19,19,Cayenne,4.9333,-52.3333,26.16,86,0,2.45,GF,2021-06-06 22:35:28
24,24,Fukuechō,32.6881,128.8419,21.18,77,100,2.82,JP,2021-06-06 22:35:37


In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Bambous Virieux,MU,24.14,-20.3428,57.7575,
4,Novo Aripuanã,BR,26.12,-5.1206,-60.3797,
7,Jamestown,US,28.62,42.0970,-79.2353,
19,Cayenne,GF,26.16,4.9333,-52.3333,
24,Fukuechō,JP,21.18,32.6881,128.8419,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
from config import p_key
params = {
    "radius": 5000,
    "types": "lodging",
    "key": p_key,
    "location": ''}
    
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

hotel_df.head()

        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Bambous Virieux,MU,24.14,-20.3428,57.7575,Casa Tia Villa
4,Novo Aripuanã,BR,26.12,-5.1206,-60.3797,Hotel Ezequiel
7,Jamestown,US,28.62,42.0970,-79.2353,Hampton Inn & Suites Jamestown
19,Cayenne,GF,26.16,4.9333,-52.3333,Hôtel Le Dronmi
24,Fukuechō,JP,21.18,32.6881,128.8419,Goto Guesthouse Utojuku & SA Car Rental


In [18]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Bambous Virieux,MU,24.14,-20.3428,57.7575,Casa Tia Villa
4,Novo Aripuanã,BR,26.12,-5.1206,-60.3797,Hotel Ezequiel
7,Jamestown,US,28.62,42.0970,-79.2353,Hampton Inn & Suites Jamestown
19,Cayenne,GF,26.16,4.9333,-52.3333,Hôtel Le Dronmi
24,Fukuechō,JP,21.18,32.6881,128.8419,Goto Guesthouse Utojuku & SA Car Rental
28,Sittwe,MM,28.57,20.1500,92.9000,Hotel Memory
30,Srīvardhan,IN,27.80,18.0333,73.0167,Savli Resort/Hotel - Shrivardhan
34,Alofi,NU,26.94,-19.0595,-169.9187,Taloa Heights
36,Vaini,TO,25.09,-21.2000,-175.2000,Keleti Beach Resort
39,Butaritari,KI,28.10,3.0707,172.7902,Isles Sunset Lodge


In [20]:
# 7. Drop the rows where there is no Hotel Name.
missing_hotels_df = hotel_df.loc[(hotel_df['Hotel Name'] == '')]
voidlist = missing_hotels_df.index
hotel_df = hotel_df.drop(voidlist)
#Conclusion: No empty hotel name in the dataframe

In [21]:
# 8a. Create the output File (CSV)
output_data_file = "vaccation_data/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [34]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °C</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig



Figure(layout=FigureLayout(height='420px'))